In [39]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

import warnings
warnings.filterwarnings( "ignore" )

sns.set_theme( style = "whitegrid" )
pio.renderers.default = "vscode"

In [41]:
experiment_dirs = {
    32: "./results/exp1",
    128: "./results/exp2",
    8192: "./results/exp3",
    16384: "./results/exp4",
}

optimizers = ["adam", "adamw", "rmsprop", "sam", "lamb", "novograd", "adopt"]

# To store data
optuna_studies = {}
cv_results_all = []
status_all = []
test_metrics_all = []

In [42]:
for bs, exp_dir in experiment_dirs.items():

    # 1) Cross validation results (single file for all optimizers)
    cv_path = os.path.join( exp_dir, "cv_results.csv" )
    if os.path.exists( cv_path ):
        df_cv = pd.read_csv( cv_path )
        df_cv["batch_size"] = bs  # Tag with batch_size
        cv_results_all.append( df_cv )

    # 2) Training status (single file that presumably has all epochs for all optimizers)
    status_path = os.path.join( exp_dir, "status.csv" )
    if os.path.exists( status_path ):
        df_status = pd.read_csv( status_path )
        df_status["batch_size"] = bs
        status_all.append( df_status )

    # 3) Test metrics (single file for all optimizers)
    test_path = os.path.join( exp_dir, "metrics.csv" )
    if os.path.exists( test_path ):
        df_test = pd.read_csv( test_path )
        df_test["batch_size"] = bs
        test_metrics_all.append( df_test )

    # 4) Optuna study: each optimizer stored in its own file named e.g. "adam_study.csv"
    for opt in optimizers:
        study_file = f"{opt}_study.csv"
        study_path = os.path.join( exp_dir, study_file )
        if os.path.exists( study_path ):
            df_study = pd.read_csv( study_path )
            # Add columns to identify batch_size and optimizer
            df_study["batch_size"] = bs
            df_study["optimizer"] = opt
            optuna_studies[ ( bs, opt ) ] = df_study

# Convert the accumulated lists into DataFrames
cv_results_all = pd.concat( cv_results_all, ignore_index = True ) if cv_results_all else pd.DataFrame()
status_all = pd.concat( status_all, ignore_index = True ) if status_all else pd.DataFrame()
test_metrics_all = pd.concat( test_metrics_all, ignore_index = True) if test_metrics_all else pd.DataFrame()

In [43]:
display(cv_results_all.head())
display(status_all.head())
display(test_metrics_all.head())

# Checking shapes
print("CV Results shape:", cv_results_all.shape)
print("Status shape:", status_all.shape)
print("Test Metrics shape:", test_metrics_all.shape)

,optimizer,train_loss,val_loss,val_accuracy,val_precisions,batch_size
0,adam,1.582576,1.592506,87.358333,0.878116,32
1,adam,1.574124,1.577685,88.841667,0.892507,32
2,adam,1.583485,1.581661,88.441667,0.890745,32
3,adam,1.584465,1.593984,87.300000,0.878437,32
4,adam,1.582678,1.600066,86.716667,0.880836,32


,optimizer,epoch,train_loss,val_loss,val_accuracy,val_precision,epoch_time,batch_size
0,adam,1,1.580819,1.738145,72.72,0.761288,10.536527,32
1,adam,2,1.578444,1.717390,74.72,0.772899,11.593730,32
2,adam,3,1.577401,1.694774,77.00,0.787239,11.657146,32
3,adam,4,1.566203,1.702925,76.64,0.781626,10.159473,32
4,adam,5,1.551694,1.649824,81.96,0.825245,11.450931,32


,optimizer,test_loss,accuracy,precision,batch_size,optimizer_config
0,adam,1.650043,81.66,0.821513,32,"lr:0.0016777375695057684, beta1:0.83539277417..."
1,adamw,1.655454,80.66,0.819796,32,"lr:0.002913773673084371, beta1:0.787163798684..."
2,rmsprop,1.710602,76.52,0.785277,32,"lr:0.0005987474910461401, alpha:0.80924363213..."
3,sam,1.682981,79.02,0.794986,32,"lr:0.09632845247360401, momentum:0.4625119749..."
4,lamb,1.630644,83.02,0.841314,32,"lr:0.004633087206038729, beta1:0.857418800290..."


CV Results shape: (140, 6)
Status shape: (560, 8)
Test Metrics shape: (28, 6)


# Validation accuracy over training epochs

In [44]:
import plotly.graph_objects as go

# Compute max points for each optimizer and batch size
max_points = {}

for (optimizer, batch_size), df_subset in status_all.groupby(["optimizer", "batch_size"]):
    max_idx = df_subset["val_accuracy"].idxmax()
    max_epoch = df_subset.loc[max_idx, "epoch"]
    max_val = df_subset.loc[max_idx, "val_accuracy"]
    min_val = df_subset["val_accuracy"].min()
    max_points[(optimizer, batch_size)] = (max_epoch, max_val, min_val)

# Create separate plots for each optimizer and batch size
figs = []

for (optimizer, batch_size), df_filtered in status_all.groupby(["optimizer", "batch_size"]):
    fig = px.line(
        df_filtered,
        x="epoch",
        y="val_accuracy",
        title=f"Validation Accuracy Over Training Epochs for {optimizer} (Batch Size {batch_size})",
        labels={"val_accuracy": "Validation Accuracy", "epoch": "Epoch"},
        width=800, height=500
    )
    
    # Adjust y-axis range based on min and max values
    max_epoch, max_val, min_val = max_points.get((optimizer, batch_size), (None, None, None))
    if max_val is not None and min_val is not None:
        fig.update_yaxes(range=[min_val - 0.05, max_val + 0.05])
    
    # Add max value marker
    if max_epoch is not None:
        fig.add_trace(go.Scatter(
            x=[max_epoch],
            y=[max_val],
            mode="markers",
            marker=dict(size=10, color="red", symbol="cross"),
            name=f"Max - {optimizer}"
        ))
    
    figs.append((fig, optimizer, batch_size))

# Display all figures
for fig, opt, batch_size in figs:
    fig.show()
    fig.write_image(f"./results/plots/training_epoch_accuracy/{opt}_{batch_size}.png")

# Training time

In [ ]:
# Make a bar plot of the average time it took for each optimizer

# Create separate plots for each batch size
batch_sizes = test_metrics_all["batch_size"].unique()
figs = []

for batch_size in batch_sizes:
    df_filtered = status_all[status_all["batch_size"] == batch_size]
    avg_time = df_filtered.groupby(['optimizer'])['epoch_time'].mean().values
    optimizer = df_filtered['optimizer'].unique()
    
    fig = px.bar(
        x=optimizer, 
        y=avg_time, 
        color=avg_time,
        barmode="group",  # Group bars side by side
        title=f"Average time Comparison for Batch Size {batch_size}",
        labels={"avg_time": "Average time", "optimizer": "Optimizer"},
        width=1000, height=600,  # Adjust figure size
        color_continuous_scale=px.colors.sequential.Bluered
    )
    figs.append((fig, optimizer, batch_size))
    fig.show()
    
# Display all figures
for fig, opt, batch_size in figs:
    fig.show()
    fig.write_image(f"./results/plots/avg_epoch_time/allop_{batch_size}.png")

In [46]:
# Create a figure showing the average training time for all batch sizes for each optimizer
avg_time = status_all.groupby(['optimizer'])['epoch_time'].mean().values
optimizer = status_all['optimizer'].unique()
    
fig = px.bar(
    x=optimizer, 
    y=avg_time, 
    color=avg_time,
    barmode="group",  # Group bars side by side
    title=f"Average time Comparison across (for all batch sizes)",
    labels={"avg_time": "Average time", "optimizer": "Optimizer"},
    width=1000, height=600,  # Adjust figure size
    color_continuous_scale=px.colors.sequential.Bluered
)

fig.show()
fig.write_image(f"./results/plots/avg_epoch_time/avg_all_batchsizes.png")

# Training loss - Validation loss - Generalization gap

In [47]:
from plotly.subplots import make_subplots
import math

In [48]:
# Convert val_loss and train_loss to numpy

status_all['gap_gen'] = status_all['val_loss'] - status_all['train_loss']

In [49]:
# Create separate plots for each optimizer and batch size
figs = []

for (optimizer, batch_size), df_filtered in status_all.groupby(["optimizer", "batch_size"]):
    
    # Reshape DataFrame to long format
    df_melted = df_filtered.melt(id_vars=["epoch"], value_vars=["train_loss", "val_loss"], 
                             var_name="Loss Type", value_name="Loss")


    # Create subplots (1 row, 2 columns)
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Training & Validation Loss Over Epochs for {optimizer} (Batch Size {batch_size})", "Generalization gap (validation loss - training loss)"))

    # First subplot (Training Loss)
    train_val_loss_fig = px.line(
        df_melted,
        x="epoch",
        y="Loss",
        color="Loss Type",
        labels={"Loss": "Loss Value", "epoch": "Epoch"}
    )
    
    # Add BOTH lines to the first subplot
    for trace in train_val_loss_fig.data:  # Loop through both traces
        fig.add_trace(trace, row=1, col=1)

    # Second subplot (Validation Loss)
    diff_loss_fig = px.line(df_filtered, x="epoch", y="gap_gen", labels={"gap_gen": "Generalization Gap"})
    diff_trace = diff_loss_fig.data[0]  # Extract trace
    
    # Modify the trace color before adding it
    diff_trace = diff_loss_fig.data[0]
    diff_trace.line.color = "green"  # Change the color (e.g., "red", "blue", "green")

    fig.add_trace(diff_trace, row=1, col=2)  # Add to subplot


    # Update layout
    fig.update_layout(height=600, width=1500)
    
    # Add x-axis labels for both subplots
    fig.update_xaxes(title_text="Epoch", row=1, col=1)  # Label for the first subplot
    fig.update_xaxes(title_text="Epoch", row=1, col=2)  # Label for the second subplot
    
    # Add y-axis label for the second subplot (Generalization Gap)
    fig.update_yaxes(title_text="Loss", row=1, col=1)  # Label for the second subplot
    fig.update_yaxes(title_text="Loss", row=1, col=2)  # Label for the second subplot
    # Show plot
    fig.show()
    
    figs.append((fig, optimizer, batch_size))

# Display all figures
for fig, opt, batch_size in figs:
    fig.show()
    fig.write_image(f"./results/plots/training_epoch_accuracy/{opt}_{batch_size}.png")

# Precision comparison over batch size for each optimizer

In [50]:
# Create separate plots for each optimizer
optimizers = test_metrics_all["optimizer"].unique()
figs = []

for optimizer in optimizers:
    df_filtered = test_metrics_all[test_metrics_all["optimizer"] == optimizer]
    df_filtered['batch_size'] = df_filtered['batch_size'].astype(str)
    fig = px.bar(
        df_filtered, 
        x="batch_size", 
        y="precision", 
        color="precision",
        barmode="group",  # Group bars side by side
        title=f"Test Precision Comparison for Optimizer {optimizer}",
        labels={"precision": "Test Precision", "batch_size": "Batch Size"},
        width=600, height=400,  # Adjust figure size
        color_continuous_scale=px.colors.sequential.Bluered
    )
    figs.append(fig)

# Display all figures
for optimizer, fig in zip( optimizers, figs ):
    fig.show()
    fig.write_image(f"./results/plots/opt_bs_test_porec/{optimizer}.png")

In [51]:
# Create separate plots for each batch size
batch_sizes = test_metrics_all["batch_size"].unique()
figs = []

for batch_size in batch_sizes:
    df_filtered = test_metrics_all[test_metrics_all["batch_size"] == batch_size]
    fig = px.bar(
        df_filtered, 
        x="optimizer", 
        y="precision", 
        color="precision",
        barmode="group",  # Group bars side by side
        title=f"Test Precision Comparison for Batch Size {batch_size}",
        labels={"precision": "Test Precision", "optimizer": "Optimizer"},
        width=1000, height=600,  # Adjust figure size
        color_continuous_scale=px.colors.sequential.Bluered
    )
    figs.append(fig)

# Display all figures
for bs, fig in zip( batch_sizes, figs ):
    fig.show()
    fig.write_image(f"./results/plots/bs_opt_test_prec/{bs}.png")

In [52]:
# Melt the DataFrame to long format for better visualization
cv_results_melted = cv_results_all.melt(id_vars=["optimizer", "batch_size"], 
                                        value_vars=["train_loss", "val_loss"], 
                                        var_name="Loss Type", 
                                        value_name="Loss")

# Create box plot to compare train_loss and val_loss for each optimizer and batch size
fig = px.box(
    cv_results_melted, 
    x="optimizer", 
    y="Loss", 
    color="Loss Type", 
    facet_col="batch_size",  # Create separate plots for each batch size
    title="Comparison of Train Loss and Validation Loss Across Optimizers and Batch Sizes",
    labels={"Loss": "Loss Value", "optimizer": "Optimizer", "batch_size": "Batch Size"},
    width=1900, height=900  # Adjust figure size
)

# Show the figure
fig.show()
fig.write_image(f"./results/plots/all_loss_variance.png")


# Precision over training epochs

In [53]:
import plotly.graph_objects as go

# Compute max points for each optimizer and batch size
max_points = {}
for (optimizer, batch_size), df_subset in status_all.groupby(["optimizer", "batch_size"]):
    max_idx = df_subset["val_precision"].idxmax()
    max_epoch = df_subset.loc[max_idx, "epoch"]
    max_val = df_subset.loc[max_idx, "val_precision"]
    min_val = df_subset["val_precision"].min()
    max_points[(optimizer, batch_size)] = (max_epoch, max_val, min_val)

# Create separate plots for each optimizer and batch size
figs = []

for (optimizer, batch_size), df_filtered in status_all.groupby(["optimizer", "batch_size"]):
    fig = px.line(
        df_filtered,
        x="epoch",
        y="val_precision",
        title=f"Validation Precision Over Training Epochs for {optimizer} (Batch Size {batch_size})",
        labels={"val_precision": "Validation Precision", "epoch": "Epoch"},
        width=800, height=500
    )
    
    # Adjust y-axis range based on min and max values
    max_epoch, max_val, min_val = max_points.get((optimizer, batch_size), (None, None, None))
    if max_val is not None and min_val is not None:
        fig.update_yaxes(range=[min_val - 0.05, max_val + 0.05])
    
    # Add max value marker
    if max_epoch is not None:
        fig.add_trace(go.Scatter(
            x=[max_epoch],
            y=[max_val],
            mode="markers",
            marker=dict(size=10, color="red", symbol="cross"),
            name=f"Max - {optimizer}"
        ))
    
    figs.append((fig, optimizer, batch_size))

# Display all figures
for fig, opt, batch_size in figs:
    fig.show()
    fig.write_image(f"./results/plots/training_epoch_precision/{opt}_{batch_size}.png")